# Lane Segmentation via BeamNG from existing map

BeamNG.tech provides an out-of-the-box solution for gathering segmentation data with special render settings for more optimal data collection. Unfortunately, the map that we attempted to use this on does not work well with default techniques (the track is one large model, meaning object annotation does not give what we want, and material annotation is currently not working with the given map). Therefore, we devised a novel procedure to collect accurate base and annotated image pairs using BeamNGpy and the world editor (F11).

**Prerequisite: Map with transparent mesh road that follows the track wanted (see below):**

Base Image:

<img src="images/track.png"  width="40%" height="40%" />


Image with Exposed Track:

<img src="images/no_track.png"  width="40%" height="40%" />

_Disclaimer: If there are updates, the package will always be more up-to-date than this notebook._


In [ ]:
# Setup
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Camera
from pathlib import Path
from datetime import datetime
import beamngpy
import matplotlib.pyplot as plt
import numpy as np
import time
import random

# Setup variables
home_dir = r'C:\Users\jazzy\Desktop\BeamNG'
user_dir = r'C:\Users\jazzy\Desktop\BeamNG'

# Scenario variables
map = 'rb_ks_monza'
car_model = 'etk800'
start_pos = (-177.105, -107.766, 154.945)
start_rot_quat = (0, 0, -0.998, 0.0598)

# BeamNG.* instance (server)
bng = BeamNGpy('localhost', 64526, home=home_dir, user=user_dir)

In [ ]:
# Start BeamNG server and simulator
bng.open()

In [ ]:
# Reconnect to an open simulator
bng.open(launch=False)

# Recording points from car

The way this workaround works is by teleporting the car randomly to a valid point, as defined by the user. One way to do this is to record points from a car's path, and then define a radius from this point such that any point within that radius is valid. The code below produces these points from a car's path, as controlled by the user.

In [ ]:
def record_points_from_car(v: Vehicle, interval=1, countdown = True):
  points = []
  if countdown:
    for i in range(5, -1, -1):
      print("Starting in " + str(i) + "...")
      time.sleep(1)
  print("Press CTRL+C to stop recording...")
  try:
    while True:
      points.append(v.get_center_of_gravity())
      time.sleep(interval)
  except KeyboardInterrupt:
    return points

In [ ]:
# Scenario for recording
scenario = Scenario(map, "record points")
car0 = Vehicle('car0', model=car_model)
scenario.add_vehicle(car0, pos=start_pos, rot_quat=start_rot_quat)

scenario.make(bng)
bng.scenario.load(scenario)
bng.scenario.start()

In [ ]:
# Record points
recorded_points = record_points_from_car(car0)

# Defining function for random point generation

The function for random point generation is custom based on the use case, but the default implementation takes the output from recording the points from the user-controlled car and finds points within a radius of any of the generated points.

_Replace this function if you want to use your own custom implementation._

In [ ]:
def generate_random_point(points, radius=4): # not optimized
  point = random.choice(points)
  offsets = (random.uniform(-radius, radius), random.uniform(-radius, radius), 0.5)
  rot_quat = beamngpy.quat.angle_to_quat((0, 0, random.uniform(-180, 180)))
  return (point[0] + offsets[0], point[1] + offsets[1], point[2] + offsets[2]), rot_quat

# Generating images

To generate images, we capture images twice: once with a clear track as a base image and one with an opaque track as an annotated image. To do this, we keep track of where we want to capture images in the simulator. We do this here by recording points off a valid path, then taking points off a radius from that path.

_This is because annotation rendering takes into account material opacity, so the track won't show up in the annotated image when the track is clear, but we need it to be to capture track details._

In [ ]:
# Number of image pairs we want to create
n = 10

# Create list of locations and directions to capture images on the track
track_points = recorded_points
locations = [generate_random_point(track_points) for _ in range(n)]

In [ ]:
# Start scenario if not started already
scenario = Scenario(map, "image generation")
car0 = Vehicle('car0', model=car_model)
scenario.add_vehicle(car0, pos=start_pos, rot_quat=start_rot_quat)

scenario.make(bng)
bng.scenario.load(scenario)
bng.scenario.start()

## Define camera setup

Define camera setup to allow for multiple cameras.

In [ ]:
camera_positions = [
    ((-0.3, 1, 2), (0, -1, 0))
] # add tuples with pos and dir [both in (x, y, z) format]

# Attach cameras to car
def attach_cameras(camera_positions, v: Vehicle, fov_y=70, near_far_planes=(0.1, 100), resolution=(1024, 1024)):
  cameras = []
  for i, (pos, dir) in enumerate(camera_positions):
    camera = Camera('camera' + str(i), bng, v, requested_update_time=-1.0, is_using_shared_memory=False,
                    pos=pos, dir=dir,
                    field_of_view_y=fov_y, near_far_planes=near_far_planes, resolution=resolution,
                    is_render_annotations=True, is_render_instance=True, is_render_depth=True)
    cameras.append(camera)
  return cameras

cameras = attach_cameras(camera_positions, car0)

## Generate base and annotated images

At this point, the mesh road defining the track should be transparent. One can do this in the world editor by setting the corresponding material's opacity to 0.

_This is done via the material alpha, make sure to enable `Alpha Clip` under `Advanced - All Layers` in the material editor._

In [ ]:
def generate_images(v: Vehicle, locations, cameras, annotated = False, verbose = True):
  images = {}
  if verbose:
    print("Generating {} total images".format(len(locations)))
  for i, (pos, dir) in enumerate(locations):
    if verbose:
      print("Grabbing image " + str(i) + "...")
    v.teleport(pos, rot_quat=dir)
    for j, cam in enumerate(cameras):
      base_image = cam.get_full_poll_request()['colour' if not annotated else 'annotation']
      name = "{}_{}".format(i, j)
      images[name] = base_image
  return images

base_images = generate_images(car0, locations, cameras)

## Generate annotated images

At this point, the mesh road defining the rack should be opaque. **Ensure that the material selected for the road has an appropriate annotation selected.** Also, ensure that the locations for the car are the same between generating base images and generating annotated images. This is how we will match base and annotated images together.

In [ ]:
annotated_images = generate_images(car0, locations, cameras, annotated = True)

## Pair images

After generating base and annotated images with the same locations and camera setup, we pair the images together to get base and labeled data.

In [ ]:
def pair_images(base: dict, annotated: dict):
  images = {}
  for name in base:
    images[name] = {'base': base[name], 'annotated': annotated[name]}
  return images

paired_images = pair_images(base_images, annotated_images)

## Save images

Save images according to date and time. All images are going to be in a folder labeled ddmmYYYYHHMMSS (date, month, year, hours, minutes, seconds). All images are labeled using the scheme `{location #}\_{camera #}\_{b for base else a for annotated}`.

_Example: 0_0_b.jpg means location 0, camera 0, base_

In the same folder, info on location and camera setup can be found in the file `setup.txt`.

The folder will be created in the present working directory (pwd). Specify where to save folder in `path` parameter relative to the pwd.


In [ ]:
def save_images(paired_images, path: str = None):
  folder_name = datetime.now().strftime("%d%m%Y%H%M%S")
  if path:
    folder = Path(path) / folder_name
  else:
    folder = Path(folder_name)
  folder.mkdir(parents = True, exist_ok = True)
  for i in paired_images:
    paired_images[i]["base"].save(folder / (i + "_b.png"))
    paired_images[i]["annotated"].save(folder / (i + "_a.png"))
  return folder_name

folder_name = save_images(paired_images)

### Examples

Base Image:

<img src="images/0_0_b.png"  width="20%" height="20%" />

Annotated Image:

<img src="images/0_0_a.png"  width="20%" height="20%" />

# Complete data collection flow

This is a shorthand for what is above without the function definitions. Use this as a quick guide / how-to-use.

In [ ]:
# Setup
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Camera
from pathlib import Path
from datetime import datetime
import beamngpy
import matplotlib.pyplot as plt
import numpy as np
import time
import random

# Setup variables
home_dir = r'C:\Users\jazzy\Desktop\BeamNG'
user_dir = r'C:\Users\jazzy\Desktop\BeamNG'

# Scenario variables
map = 'rb_ks_monza'
car_model = 'etk800'
start_pos = (-177.105, -107.766, 154.945)
start_rot_quat = (0, 0, -0.998, 0.0598)

# BeamNG.* instance (server)
bng = BeamNGpy('localhost', 64526, home=home_dir, user=user_dir)
bng.open()

# Scenario for recording
scenario = Scenario(map, "record points")
car0 = Vehicle('car0', model=car_model)
scenario.add_vehicle(car0, pos=start_pos, rot_quat=start_rot_quat)
scenario.make(bng)
bng.scenario.load(scenario)
bng.scenario.start()

# wait
input("Press Enter to start recording...")

# Record points
recorded_points = record_points_from_car(car0)

# Create list of locations and directions to capture images on the track
n = 10
locations = [generate_random_point(recorded_points) for _ in range(n)]

# Attach cameras to car
camera_positions = [
    ((-0.3, 1, 2), (0, -1, 0))
]
cameras = attach_cameras(camera_positions, car0)

# Generate base images
base_images = generate_images(car0, locations, cameras)

# wait
input("Switch track for annotation mode. Press Enter when complete...")

# Generate annotated images (MAKE SURE TO SWITCH TRACK)
annotated_images = generate_images(car0, locations, cameras, annotated = True)

# Pair images
paired_images = pair_images(base_images, annotated_images)

# Save images
folder_name = save_images(paired_images)

print("Images saved in folder {}".format(folder_name))